# Locating gauge stations on MSC model grids



This tutorial demonstrates how to find gauge stations on the NSRPS model grid using the Web Coverage Service (WCS). If this is your first time accessing GeoMet's Web Coverage Service, we recommend first reading the geomet-auth-wcs-quick-demo tutorial. For more detailed information about accessing secured layers with the Web Map Service and Web Coverage Service we recommend reading our more comprehensive tutorials, geomet-auth-wms-examples and geomet-auth-wms-examples.

## General requirements

In [2]:
# data
import warnings
import re
import time
import configparser
from datetime import datetime, timedelta
import xarray as xr 
import pandas as pd
import numpy as np
from shapely.geometry import Point

# web map services 
from owslib.wms import WebMapService
from owslib.wcs import WebCoverageService
from owslib.wcs import Authentication
from owslib.ogcapi.features import Features

# plotting
#import holoviews
#import hvplot.xarray
#import hvplot.pandas

In [3]:
# loading login information
config = configparser.ConfigParser()
config.read_file(open('config.cfg')) 

login = config['Login']

## Requesting latest DHPS predictions

In [25]:
# Deterministic Hydrologic Prediction System (DHPS) is a sub-system of the NSRPS
layer_name = 'DHPS-Analysis_1km_RiverDischarge'

In [26]:
# first querying the WMS for time metadata
wms = WebMapService(f'https://geo.weather.gc.ca/geomet?&SERVICE=WMS&LAYERS={layer_name}',
                    version='1.3.0',
                    auth=Authentication(username=login['Username'], password=login['Password']),
                    timeout=300)

In [34]:
first_datetime, last_datetime, datetime_interval = wms[layer_name].dimensions['time']['values'][0].split('/')
#oldest_fcast, newest_fcast, issue_interval = wms[layer_name].dimensions['reference_time']['values'][0].split('/')
#first_datetime, last_datetime, datetime_interval = wms[layer_name].dimensions['reference_time']['values'][0].split('/')
print(wms[layer_name].dimensions['time']['values'][0].split('/'))
print(wms[layer_name].dimensions['reference_time']['values'][0].split('/'))

['2024-01-26T12:00:00Z', '2024-01-26T12:00:00Z', 'PT0H']
['2024-01-25T01:00:00Z', '2024-01-26T12:00:00Z', 'PT1H']


In [32]:
iso_format = "%Y-%m-%dT%H:%M:%SZ"

# convert dates to datetime objects
first = datetime.strptime(first_datetime, iso_format)
last = datetime.strptime(last_datetime, iso_format)

# remove anything that isn't a number from the datetime interval (time between forecasts)
intvl = int(re.sub(r'\D', '', datetime_interval))

# create a list of forecast datetimes (we will add these to the requested data)
fcasthrs = [first]
while first < last:
    first = first + timedelta(hours=intvl)
    fcasthrs.append(first)

# create a list of iso formatted forecast datetime strings (we will use these in the WCS requests)
fcasthrs_str = [datetime.strftime(hr, iso_format) for hr in fcasthrs]
print(fcasthrs_str)

['2024-01-26T12:00:00Z']


Note: the next cell may take a few minutes to run because several requests are being made.

In [33]:
wcs = WebCoverageService(f'https://geo.weather.gc.ca/geomet?&SERVICE=WCS&COVERAGEID={layer_name}', 
                    auth=Authentication(username=login['Username'], password=login['Password']),
                    version='2.0.1',
                    timeout=300
                    )

# local time
time_zone = -7

# for each forecast hour, make a WCS request
arrys = []

for i, hr in enumerate(fcasthrs_str):
    response = wcs.getCoverage(identifier = [layer_name], 
                        format = 'image/netcdf', 
                        subsettingcrs = 'EPSG:4326', 
                        subsets = [('lat', 50, 52.0), ('lon', -117.0, -113.0)],
                        DIM_REFERENCE_TIME=newest_fcast, 
                        TIME=hr 
                       )
    
    # read into an xarray
    ds = xr.open_dataset(response.read()).load()
    
    # add the time metadata as a new dimension and coordinate
    ds = ds.expand_dims(time=[fcasthrs[i] + timedelta(hours=time_zone)])
    
    # append to list of xarrays
    arrys.append(ds)
    
fcasts = xr.concat([ds for ds in arrys], dim='time')


In [14]:
fcasts.to_netcdf(f'{layer_name}_{first}.nc')


In [11]:
fcasts.to_netcdf('river_discharge.nc')

## Finding stations on the DHPS model grid

In other tutorials, we've looked at the gridded DHPS data, but we may be more interested in the streamflow forecasts at specific stations. As an example, let's say we are interested in the flows at [Sturgeon River at McDougall Mills](https://wateroffice.ec.gc.ca/report/real_time_e.html?stn=05QA004). The following example demonstrates how we can extract the river discharge forecasts at our station of interest.


Note: the station latitutdes and longitudes in the models (DHPS/EHPS/WCPS) may not be exactly the same as the actual stations' latitudes and longitudes. In some cases, the modellers have had to slightly adjust the stations' latitudes/longitudes to minimize drainage area errors. This is a by-product of creating a gridded stream network. As a result, to find the stations of interest on the model grid we need to know what latitudes/longitudes the modellers have assigned to each station. In this example, Sturgeon River at McDougall Mills has an actual lat/lon of 50.167222, -91.540556 (in decimal degrees). In the model, Sturgeon River has a lat/lon of 50.1701, -91.5418. In addition, the model's station lat/lon may not align perfectly with the lat/lon axes of the model grid. The lat/lon axes of the model grid specify the center point of each grid cell and a station's model location may not be located in the center of its grid cell. Therefore, to find the grid cell that contains our station of interest we need to identify the closest lat/lon center point to the model's station lat/lon.

In [9]:
# Sturgeon River at McDougall Mills

# function to relate the CCMEP station location to its grid location
def find_stn_on_grid(latstn, lonstn):
    
    # find grid lat
    closest_lat_idx = (np.abs(fcasts.lat.data - latstn)).argmin()
    lat = fcasts.lat.data[closest_lat_idx]
    
    # find grid lon
    closest_lon_idx = (np.abs(fcasts.lon.data - lonstn)).argmin()
    lon = fcasts.lon.data[closest_lon_idx]
    
    return lat, lon

# CCMEP station locations
latst = 50.1701 #48.85
lonst = -91.5418 #92.723611

# find the station on the grid
lat, lon = find_stn_on_grid(latst, lonst)

In [10]:
# compare the CCMEP station location and the grid station location
ll_info = f"CCMEP lat/lon ({str(latst)}, {str(lonst)}) | Grid lat/lon ({str(np.round(lat,4))}, {str(np.round(lon,4))})"
ll_info

'CCMEP lat/lon (50.1701, -91.5418) | Grid lat/lon (50.1708, -91.5458)'

In [11]:
# plot the data
station = fcasts.sel(lat=lat, lon=lon)
station.hvplot(title='Sturgeon River at McDougall Mills | '+ll_info, ylabel='River Discharge (m3/s)', width=900)

:Curve   [time]   (Band1)